In [2]:
import pandas as pd
import numpy as np
import pyodbc as db
import os
import glob
from glob import glob
import re
import luigi
from operator import itemgetter
from persiantools import characters
from persiantools.jdatetime import JalaliDate
import persiantools
import datetime
from re import search
import itertools
from statistics import mode
import datetime as dt
from joblib import Parallel, delayed
import chardet
import requests
from bs4 import BeautifulSoup
import json
import time
import io
import jdatetime
import TSETMCScraping
import ar_to_fa
import clean_cols
import str_to_date
import str_to_jdate
import str_to_time
import cleaning_path
from luigi.mock import MockTarget
import stock_information_cleaning

Stock_Information

In [ ]:
class stock_information(luigi.Task):
    read_path= luigi.Parameter(default=None)
    save_path= luigi.Parameter(default=None)
    pipeline_info= luigi.Parameter(default=None)
    def output(self):
        return luigi.LocalTarget(self.save_path),luigi.LocalTarget(self.pipeline_info)
    def requiers(self):
        return None
    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # save_path_corr= re.sub("output.csv$",f"\\{str(jdatetime.date.today())}.csv",self.save_path)
        with open(self.pipeline_info,"a") as f:
            f.write(f"\nRunning stock_information at {str(datetime.date.today())} , {nowtime}")
        if os.path.exists(self.save_path):
            with open(self.pipeline_info,"a") as f:
                f.write(f"\n{str(datetime.date.today())}: there is the file with the same data")
        else:
            scrap_links= pd.read_excel(self.read_path)
            df= TSETMCScraping.stock_information(scrap_links)
            df1= df.copy()
            with open(self.pipeline_info,"a") as f:
                f.write(f'\n{str(datetime.date.today())}, {nowtime}: Running stock_information is SUCCESSFULL')
            try:
                string_column=['company_name','stock_name','stock_persian_30','market','name_sector',
                                'name_subsector','stock_name_2','trade_purpose','CEO','auditor']
                price_column=['capital']
                code_column=['code_stock_12','code_stock_5','code_company_4','code_company_12','code_tablo',
                                'code_sector','code_subsector','shenaseh_meli']
                df1= stock_information_cleaning.string_clean(string_column)
                df1= stock_information_cleaning.price_clean(price_columns=price_column)
                df1= stock_information_cleaning.code_clean(code_columns=code_column)
                df1.to_csv(self.save_path, encoding='utf-8-sig', index=False)
                with open(self.pipeline_info,"a") as f:
                    f.write(f'\n{str(datetime.date.today())}, {nowtime}: Cleanning is SUCCESSFULL')
                with open(self.pipeline_info,"a") as f:
                    f.write('\n-------------------------------------------------------------')
            except:
                df.to_csv(self.save_path, encoding='utf-8-sig', index=False)
                with open(self.pipeline_info,"a") as f:
                    f.write(f'\n{str(datetime.date.today())}, {nowtime}: Cleanning is FAILED')
                with open(self.pipeline_info,"a") as f:
                    f.write('\n-------------------------------------------------------------')

In [ ]:
class stock_info_DB(luigi.Task):
    save_path= luigi.Parameter(default=None)
    pipeline_info= luigi.Parameter(default=None)
    def requiers(self):
        return stock_information(self.save_path)
    def output(self):
        return luigi.LocalTarget(self.pipeline_info)
    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # موارد خالی برای نام سرور، اسم دیتابیس، نام کاربری و رمز عبور باید تکمیل گردد
        server_name=''
        DataBase_name=''
        username=''
        password=''
        cnxn= db.connect('DRIVER={SQL Server};SERVER='+server_name+';DATABASE='+DataBase_name+';UID'+username+';PWD='+password)
        cursor= cnxn.cursor()
        information_path= self.input().path
        df= pd.read_csv(information_path)
        try:
            for index, row in df.iterrows():
                    cursor.execute('''
                                INSERT INTO stock_info ('code_stock_12','code_stock_5','company_name_English','code_company_4','company_name','stock_name','stock_persian_30',
                                'code_company_12','market','code_tablo','code_sector','name_sector','code_subsector','name_subsector',
                                'stock_name_2','trade_purpose','CEO','address','phone_number','FAX','offic_address',
                                'shareholder_affairs_address','website','email','auditor','capital','year','financial_manager',
                                'shenaseh_meli','stock_code_TSETMC')
                                VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                                ''',
                                row.loc['code_stock_12'], 
                                row.loc['code_stock_5'],
                                row.loc['company_name_English'],
                                row.loc['code_company_4'],
                                row.loc['company_name'],
                                row.loc['stock_name'],
                                row.loc['stock_persian_30'],
                                row.loc['code_company_12'],
                                row.loc['market'],
                                row.loc['code_tablo'],
                                row.loc['code_sector'],
                                row.loc['name_sector'],
                                row.loc['code_subsector'],
                                row.loc['name_subsector'],
                                row.loc['stock_name_2'],
                                row.loc['trade_purpose'],
                                row.loc['CEO'],
                                row.loc['address'],
                                row.loc['phone_number'],
                                row.loc['FAX'],
                                row.loc['offic_address'],
                                row.loc['shareholder_affairs_address'],
                                row.loc['website'],
                                row.loc['email'],
                                row.loc['auditor'],
                                row.loc['capital'],
                                row.loc['year'],
                                row.loc['financial_manager'],
                                row.loc['shenaseh_meli'],
                                row.loc['stock_code_TSETMC']
                                )
            cnxn.commit()
            cursor.close()
            with open(self.pipeline_info, 'a') as f:
                f.write(f"{str(datetime.date.today())} ,{nowtime}: Insert into DB is SUCCESSFULL")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')
        except:
            with open(self.pipeline_info,'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Insert into DB is FAILED")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')

In [ ]:
class remove_dup_stoc_info(luigi.Task):
    pipeline_info= luigi.Parameter(default=None)
    def requiers(self):
        return stock_info_DB()
    def output(self):
        return luigi.LocalTarget(self.pipeline_info)
    
    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # موارد خالی برای نام سرور، اسم دیتابیس، نام کاربری و رمز عبور باید تکمیل گردد
        server_name=''
        DataBase_name=''
        username=''
        password=''
        cnxn= db.connect('DRIVER={SQL Server};SERVER='+server_name+';DATABASE='+DataBase_name+';UID'+username+';PWD='+password)
        cursor= cnxn.cursor()
        try:
            cursor.execute('''          
                    select distinct code_stock_12, code_stock_5, company_name_English, code_company_4, company_name, stock_name, stock_persian_30,
                     code_company_12, market, code_tablo, code_sector, name_sector, code_subsector, name_subsector, stock_name_2, trade_purpose,
                    CEO,address, phone_number, FAX, offic_address, shareholder_affairs_address, website, email,
                    auditor, capital, year, financial_manager, shenaseh_meli, stock_code_TSETMC from stock_info ''')
            cnxn.commit()
            cursor.close()
            with open(self.pipeline_info, 'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Remove Duplicates from DB is SUCCESSFULL")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')
        except:
            with open(self.Pipelines_log,'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Remove Duplicates from DB is FAILED")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')

In [ ]:
if __name__=='__main__':
    luigi.build([remove_dup_stoc_info(read_path="C:\\Users\\ah.roudsaz\\Desktop\\test.xlsx",
                save_path=f"C:\\Users\\ah.roudsaz\\Desktop\\test\\{str(jdatetime.date.today())}.csv",pipeline_info="C:\\Users\\ah.roudsaz\\Desktop\\test\\Pipeline_log.txt")],local_scheduler=True)

Stock_Trade_History

In [ ]:
class stock_trade(luigi.Task):
    pipeline_info= luigi.Parameter(default=None)
    stock_trade= luigi.Parameter(default=None)
    read_file= luigi.Parameter(default=None)

    def requiers(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.stock_trade),luigi.LocalTarget(self.pipeline_info)

    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # save_path= re.sub("output.csv$",f"\\{str(jdatetime.date.today())}.csv",self.stock_trade)
        with open(self.pipeline_info,"a") as f:
            f.write(f"\nRunning trade scrap at {str(datetime.date.today())}")
        if os.path.exists(self.stock_trade):
            with open(self.pipeline_info,"a") as f:
                f.write(f"\n{str(datetime.date.today())}, {nowtime}: there is the file with the same data")
        else:
            scrap_links= pd.read_excel(self.read_file)
            df= TSETMCScraping.trade_history(scrap_links)
            df1= df.copy()
            with open(self.pipeline_info,"a") as f:
                f.write(f'\n{str(datetime.date.today())}, {nowtime}: Running trade scrap is SUCCESSFULL')
            try:
                string_column= ['']
                code_column=['']
                price_column=['']
                df1= stock_information_cleaning.string_clean(string_columns=string_column)
                df1= stock_information_cleaning.price_clean(price_columns=price_column)
                df1= stock_information_cleaning.code_clean(code_columns=code_column)
                df1.to_csv(self.stock_trade, encoding='utf-8-sig', index=False)
                with open(self.pipeline_info,"a") as f:
                    f.write(f'\n{str(datetime.date.today())}, {nowtime}: Cleanning is SUCCESSFULL')
                with open(self.pipeline_info,"a") as f:
                    f.write('\n-------------------------------------------------------------')
            except:
                df.to_csv(self.stock_trade, encoding='utf-8-sig', index=False)
                with open(self.pipeline_info,"a") as f:
                    f.write(f'\n{str(datetime.date.today())}, {nowtime}: Cleanning is FAILED')
                with open(self.pipeline_info,"a") as f:
                    f.write('\n-------------------------------------------------------------')

In [ ]:
class stock_trade_DB(luigi.Task):
    stock_trade= luigi.Parameter(default=None)
    pipeline_info= luigi.Parameter(default=None)
    def requiers(self):
        return stock_trade(self.stock_trade)
    def output(self):
        return luigi.LocalTarget(self.pipeline_info)
    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # موارد خالی برای نام سرور، اسم دیتابیس، نام کاربری و رمز عبور باید تکمیل گردد
        server_name=''
        DataBase_name=''
        username=''
        password=''
        cnxn= db.connect('DRIVER={SQL Server};SERVER='+server_name+';DATABASE='+DataBase_name+';UID'+username+';PWD='+password)
        cursor= cnxn.cursor()
        information_path= self.input().path
        df= pd.read_csv(information_path)
        try:
            for index, row in df.iterrows():
                    cursor.execute('''
                            INSERT INTO trade_history ('insCode','dEven','nTran','hEven','qTitTran','pTran','qTitNgJ','iSensVarP','pPhSeaCotJ','pPbSeaCotJ',
                            'iAnuTran','xqVarPJDrPRf','canceledCompony','shenaseh_meli','Date')
                            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)
                            ''',
                            row.loc['insCode'], 
                            row.loc['dEven'],
                            row.loc['nTran'],
                            row.loc['hEven'],
                            row.loc['qTitTran'],
                            row.loc['pTran'],
                            row.loc['qTitNgJ'],
                            row.loc['iSensVarP'],
                            row.loc['pPhSeaCotJ'],
                            row.loc['pPbSeaCotJ'],
                            row.loc['iAnuTran'],
                            row.loc['xqVarPJDrPRf'],
                            row.loc['canceledCompony'],
                            row.loc['shenaseh_meli'],
                            row.loc['Date']
                            )
            cnxn.commit()
            cursor.close()
            with open(self.pipeline_info, 'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Insert into DB is SUCCESSFULL")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')
        except:
            with open(self.Pipelines_log,'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Insert into DB is FAILED")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')

In [ ]:
class remove_dup_trade(luigi.Task):
    pipeline_info= luigi.Parameter(default=None)
    def requiers(self):
        return stock_trade_DB()
    def output(self):
        return luigi.LocalTarget(self.pipeline_info)
    
    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # موارد خالی برای نام سرور، اسم دیتابیس، نام کاربری و رمز عبور باید تکمیل گردد
        server_name=''
        DataBase_name=''
        username=''
        password=''
        cnxn= db.connect('DRIVER={SQL Server};SERVER='+server_name+';DATABASE='+DataBase_name+';UID'+username+';PWD='+password)
        cursor= cnxn.cursor()
        try:
            cursor.execute('''          
                    select distinct insCode, dEven, nTran, Change, hEven, qTitTran, pTran, qTitNgJ, iSensVarP,
                    pPhSeaCotJ, pPbSeaCotJ, iAnuTran, xqVarPJDrPRf, canceledCompony, shenaseh_meli, Date from trade_history ''')
            cnxn.commit()
            cursor.close()
            with open(self.pipeline_info, 'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Remove Duplicates from DB is SUCCESSFULL")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')
        except:
            with open(self.Pipelines_log,'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Remove Duplicates from DB is FAILED")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')

In [ ]:
if __name__=='__main__':
    luigi.build([remove_dup_trade(read_path="C:\\Users\\ah.roudsaz\\Desktop\\test.xlsx",
                stock_trade=f"C:\\Users\\ah.roudsaz\\Desktop\\test\\{str(jdatetime.date.today())}.csv",pipeline_info="C:\\Users\\ah.roudsaz\\Desktop\\test\\Pipeline_log.txt")],local_scheduler=True)

ShareHolders

In [ ]:
class shareholders(luigi.Task):
    pipeline_info= luigi.Parameter(default=None)
    shareholders_file= luigi.Parameter(default=None)
    read_file= luigi.Parameter(default=None)

    def requiers(self):
        return None
    def output(self):
        return luigi.LocalTarget(self.shareholders_file),luigi.LocalTarget(self.pipeline_info)

    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # save_path= re.sub("output.csv$",f"\\{str(jdatetime.date.today())}.csv",self.shareholders_file)
        with open(self.pipeline_info,"a") as f:
            f.write(f"\nRunning shareholders at {str(datetime.date.today())}")
        if os.path.exists(self.shareholders_file):
            with open(self.pipeline_info,"a") as f:
                f.write(f"\n{str(datetime.date.today())}, {nowtime}: there is the file with the same data")
        else:
            scrap_links= pd.read_excel(self.read_file)
            df= TSETMCScraping.trade_history(scrap_links)
            df1= df.copy()
            with open(self.pipeline_info,"a") as f:
                f.write(f'\n{str(datetime.date.today())}, {nowtime}: Running shareholders is SUCCESSFULL')
            try:
                string_column= ['']
                code_column=['']
                price_column=['']
                df1= stock_information_cleaning.string_clean(string_columns=string_column)
                df1= stock_information_cleaning.price_clean(price_columns=price_column)
                df1= stock_information_cleaning.code_clean(code_columns=code_column)
                df1.to_csv(self.shareholders_file, encoding='utf-8-sig', index=False)
                with open(self.pipeline_info,"a") as f:
                    f.write(f'\n{str(datetime.date.today())}, {nowtime}: Cleanning is SUCCESSFULL')
                with open(self.pipeline_info,"a") as f:
                    f.write('\n-------------------------------------------------------------')
            except:
                df.to_csv(self.shareholders_file, encoding='utf-8-sig', index=False)
                with open(self.pipeline_info,"a") as f:
                    f.write(f'\n{str(datetime.date.today())}, {nowtime}: Cleanning is FAILED')
                with open(self.pipeline_info,"a") as f:
                    f.write('\n-------------------------------------------------------------')

In [ ]:
class shareholders_DB(luigi.Task):
    shareholders_file= luigi.Parameter(default=None)
    pipeline_info= luigi.Parameter(default=None)
    def requiers(self):
        return shareholders(self.shareholders_file)
    def output(self):
        return luigi.LocalTarget(self.pipeline_info)
    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # موارد خالی برای نام سرور، اسم دیتابیس، نام کاربری و رمز عبور باید تکمیل گردد
        server_name=''
        DataBase_name=''
        username=''
        password=''
        cnxn= db.connect('DRIVER={SQL Server};SERVER='+server_name+';DATABASE='+DataBase_name+';UID'+username+';PWD='+password)
        cursor= cnxn.cursor()
        information_path= self.input().path
        df= pd.read_csv(information_path)
        try:
            for index, row in df.iterrows():
                    cursor.execute('''
                            INSERT INTO shareholders ('Shareholder','Share','Percent','Change','Compony','shenaseh_meli','shareholder_id')
                            VALUES (?,?,?,?,?,?,?)
                            ''',
                            row.loc['Shareholder'], 
                            row.loc['Share'],
                            row.loc['Percent'],
                            row.loc['Change'],
                            row.loc['Compony'],
                            row.loc['shenaseh_meli'],
                            row.loc['shareholder_id']
                            )
            cnxn.commit()
            cursor.close()
            with open(self.pipeline_info, 'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Insert into DB is SUCCESSFULL")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')
        except:
            with open(self.pipeline_info,'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Insert into DB is FAILED")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')

In [3]:
class remove_dup_share(luigi.Task):
    pipeline_info= luigi.Parameter(default=None)
    def requiers(self):
        return shareholders_DB()
    def output(self):
        return luigi.LocalTarget(self.pipeline_info)
    
    def run(self):
        now= jdatetime.datetime.now()
        nowtime= now.strftime("%H:%M:%S")
        # موارد خالی برای نام سرور، اسم دیتابیس، نام کاربری و رمز عبور باید تکمیل گردد
        server_name=''
        DataBase_name=''
        username=''
        password=''
        cnxn= db.connect('DRIVER={SQL Server};SERVER='+server_name+';DATABASE='+DataBase_name+';UID'+username+';PWD='+password)
        cursor= cnxn.cursor()
        try:
            cursor.execute('''select distinct Shareholder, Share, Percent, Change, Compony, shenaseh_meli, shareholder_id from shareholders ''')
            cnxn.commit()
            cursor.close()
            with open(self.pipeline_info, 'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Remove Duplicates from DB is SUCCESSFULL")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')
        except:
            with open(self.pipeline_info,'a') as f:
                f.write(f"{str(datetime.date.today())}, {nowtime}: Remove Duplicates from DB is FAILED")
            with open(self.pipeline_info,"a") as f:
                f.write('\n-------------------------------------------------------------')

In [ ]:
if __name__=='__main__':
    luigi.build([remove_dup_share(read_path="C:\\Users\\ah.roudsaz\\Desktop\\test.xlsx",
                shareholders_file=f"C:\\Users\\ah.roudsaz\\Desktop\\test\\{str(jdatetime.date.today())}.csv",pipeline_info="C:\\Users\\ah.roudsaz\\Desktop\\test\\Pipeline_log.txt")],local_scheduler=True)